In [ ]:
import cv2
import numpy as np
import tensorflow as tf

In [ ]:
SHAPE = (224, 224, 3)
TRAIN_DIR = "data/training_set/"
TEST_DIR = "data/test_set/"

In [ ]:
# Model functions
def input_layer(input_shape: tuple=SHAPE):
    return tf.keras.Input(shape=input_shape)

def conv_block(x, nb_filters: int=64, nb_layers: int=2, repeat: int=-1):
    for layer in range(1, nb_layers+1):    
        layer_name = f"conv3_{nb_filters}_{layer}"
        if repeat != -1:
            layer_name = f"conv3_{nb_filters}_{layer}_{repeat}"
        x = tf.keras.layers.Conv2D(
            filters=nb_filters,
            kernel_size=(3, 3),
            padding='same',
            activation='relu',
            name=layer_name,
        )(x)
        
    x = tf.keras.layers.MaxPooling2D(
        pool_size=2,
        strides=2,
        padding='same',
    )(x)
    return x

def fc_block(x, layer_sizes=[512, 256, 2]):
    x = tf.keras.layers.Flatten()(x)
    for layer in layer_sizes[:-1]:
        x = tf.keras.layers.Dense(
            units=layer, 
            activation='relu', 
        )(x)
    
    output = tf.keras.layers.Dense(
            units=layer_sizes[-1], 
            activation='softmax', 
            name=f"output-layer-{layer_sizes[-1]}"
    )(x)
    return output

In [ ]:
inp = input_layer()
x = conv_block(inp, 64, 1)
x = conv_block(x, 128, 1)
x = conv_block(x, 256, 2)
x = conv_block(x, 512, 2)
x = conv_block(x, 756, 2)
out = fc_block(x)
vgg_a = tf.keras.Model(inputs=inp, outputs=out)
vgg_a.summary()

In [ ]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.)
test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.)

train = train_gen.flow_from_directory(TRAIN_DIR)
test = test_gen.flow_from_directory(TEST_DIR)

In [ ]:
vgg_a.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
vgg_a.fit(train, epochs=2, validation_data=test)